In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
(xtrain, xtest), (ytrain, ytest) = tf.keras.datasets.cifar10.load_data()

In [0]:
xtrain = xtrain.astype("float32")
xtest = xtest.astype("float32")

xtrain= xtrain/255.0
xtest = xtest/255.0

In [0]:
ytrain = tf.keras.utils.to_categorical(ytrain)
ytest = tf.keras.utils.to_categorical(ytest)

In [0]:
def inception_model(width, height, depth, num_classes):
  
  inputs = tf.keras.Input(shape=(width, height, depth))

  tower1 = tf.keras.layers.Conv2D(64, (1, 1), padding="same", activation="relu")(inputs)
  tower1 = tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu")(tower1)

  tower2 = tf.keras.layers.Conv2D(64, (1, 1), padding="same", activation="relu")(tower1)
  tower2 = tf.keras.layers.Conv2D(64, (5, 5), padding="same", activation="relu")(tower2)

  tower3 = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding="same")(tower2)
  tower3 = tf.keras.layers.Conv2D(64, (1, 1), padding="same", activation="relu")(tower3)

  output = tf.keras.layers.concatenate([tower1, tower2, tower3], axis=3)

  output = tf.keras.layers.Flatten()(output)
  output = tf.keras.layers.Dense(num_classes, activation="softmax")(output)

  model = tf.keras.Model(inputs=inputs, outputs=output, name="inception_model")
  return model

In [0]:
model = inception_model(32, 32, 3, 10)
model.summary()

W0714 19:58:43.021866 140140499781504 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "inception_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   36928       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 64)   4160        conv2d_1[0][0]                   
____________________________________________________________________________________

In [0]:
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = tf.keras.optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
history = model.fit(xtrain, xtest, batch_size=32, epochs=epochs)